# Population data scrap and clean

In [1]:
import time
import pickle
from selenium import webdriver
from bs4 import BeautifulSoup
import os

### 1. Use selenium to scroll to the bottom of the webpage and download the HTML code

In [10]:
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)
driver.get('https://data.worldbank.org/indicator/SP.POP.TOTL?end=2001&start=2000')
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")


In [11]:
html_source = driver.page_source
pop_data = html_source.encode('utf-8')

In [13]:
pickle.dump(pop_data, open('pop_data.pickle', 'wb'))

### 2. Use BeautifulSoup to extract each country's name with the corresponding population in 2000 and 2001

In [14]:
pop_soup = BeautifulSoup(pop_data, 'lxml')

In [15]:
country_objects = pop_soup.find(class_='item').findNextSibling()
rows = []
for element in country_objects:
    print(element, '\n')
    rows.append(element)

<div class="item" style="min-height: 49px;"><div>Afghanistan</div><div>20,093.76</div><div>20,966.46</div><div></div></div> 

<div class="item" style="min-height: 49px;"><div>Albania</div><div>3,089.03</div><div>3,060.17</div><div></div></div> 

<div class="item" style="min-height: 49px;"><div>Algeria</div><div>31,183.66</div><div>31,592.15</div><div></div></div> 

<div class="item" style="min-height: 49px;"><div>American Samoa</div><div>57.52</div><div>58.17</div><div></div></div> 

<div class="item" style="min-height: 49px;"><div>Andorra</div><div>65.39</div><div>67.34</div><div></div></div> 

<div class="item" style="min-height: 49px;"><div>Angola</div><div>16,440.92</div><div>16,983.27</div><div></div></div> 

<div class="item" style="min-height: 49px;"><div>Antigua and Barbuda</div><div>83.58</div><div>85.06</div><div></div></div> 

<div class="item" style="min-height: 49px;"><div>Argentina</div><div>37,057.45</div><div>37,471.51</div><div></div></div> 

<div class="item" style="m

In [16]:
rows[0]
type(rows[0])

bs4.element.Tag

In [17]:
rows[0]

<div class="item" style="min-height: 49px;"><div>Afghanistan</div><div>20,093.76</div><div>20,966.46</div><div></div></div>

In [18]:
country = rows[0].find('div')
clean_country = country.text
clean_country

'Afghanistan'

In [19]:
yr2000 = country.findNextSibling()
yr2000_clean = yr2000.text
yr2000_clean

'20,093.76'

In [20]:
yr2001 = yr2000.findNextSibling()
yr2001_clean = yr2001.text
yr2001_clean

'20,966.46'

In [21]:
country_list = []
yr2000_list = []
yr2001_list = []
for row in rows:
    country = row.find('div')
    clean_country = country.text.strip()
    country_list.append(clean_country)
    
    yr2000 = country.findNextSibling()
    yr2000_clean = yr2000.text.replace(',', '').replace('.', '')
    yr2000_list.append(yr2000_clean)
    
    yr2001 = yr2000.findNextSibling()
    yr2001_clean = yr2001.text.replace(',', '').replace('.', '')
    yr2001_list.append(yr2001_clean)
    

### 3. Create a pandas dataframe from scraped data

In [23]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [24]:
d = {'Country':country_list, '2000':yr2000_list, '2001':yr2001_list}
df = pd.DataFrame(d)
df.tail(5)

,Country,2000,2001
243,Low & middle income,500667253,507870517
244,Low income,46563133,47847797
245,Lower middle income,228023522,232003580
246,Middle income,454104120,460022720
247,Upper middle income,226080598,228019140


In [25]:
df = df.iloc[:217]
df.rename(columns={'Country':'Country Name'}, inplace=True)
df.head()

,Country Name,2000,2001
0,Afghanistan,2009376,2096646
1,Albania,308903,306017
2,Algeria,3118366,3159215
3,American Samoa,5752,5817
4,Andorra,6539,6734


In [26]:
with open('Pop2000_2001_df', 'wb') as to_write:
    pickle.dump(df, to_write)

In [27]:
pop_df1 = df

### 4. Download outstanding population data and merge dataframes by country name

In [28]:
pop_df2 = pd.read_csv('/Users/leesurkis/Documents/project2prep/Population.csv', header=1)
pop_df2.head()

,Country Name,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Aruba,94992.0,97017.0,98737.0,100031.0,100832.0,101220.0,101353.0,101453.0,101669.0,102053.0,102577.0,103187.0,103795.0,104341.0
1,Afghanistan,21979923.0,23064851.0,24118979.0,25070798.0,25893450.0,26616792.0,27294031.0,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0,32758020.0,33736494.0
2,Angola,17572649.0,18203369.0,18865716.0,19552542.0,20262399.0,20997687.0,21759420.0,22549547.0,23369131.0,24218565.0,25096150.0,25998340.0,26920466.0,27859305.0
3,Albania,3051010.0,3039616.0,3026939.0,3011487.0,2992547.0,2970017.0,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0
4,Andorra,70049.0,73182.0,76244.0,78867.0,80991.0,82683.0,83861.0,84462.0,84449.0,83751.0,82431.0,80788.0,79223.0,78014.0


In [29]:
totalpop_df = pd.merge(pop_df1, pop_df2, on='Country Name')
totalpop_df.head()

,Country Name,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Afghanistan,2009376,2096646,21979923.0,23064851.0,24118979.0,25070798.0,25893450.0,26616792.0,27294031.0,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0,32758020.0,33736494.0
1,Albania,308903,306017,3051010.0,3039616.0,3026939.0,3011487.0,2992547.0,2970017.0,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0
2,Algeria,3118366,3159215,31995046.0,32403514.0,32831096.0,33288437.0,33777915.0,34300076.0,34860715.0,35465760.0,36117637.0,36819558.0,37565847.0,38338562.0,39113313.0,39871528.0
3,American Samoa,5752,5817,58731.0,59117.0,59264.0,59118.0,58650.0,57903.0,57030.0,56227.0,55637.0,55320.0,55230.0,55307.0,55437.0,55537.0
4,Andorra,6539,6734,70049.0,73182.0,76244.0,78867.0,80991.0,82683.0,83861.0,84462.0,84449.0,83751.0,82431.0,80788.0,79223.0,78014.0


In [67]:
totalpop_df.T.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215
Country Name,Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Aruba,Australia,Austria,Azerbaijan,"Bahamas, The",Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bermuda,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,British Virgin Islands,Brunei Darussalam,Bulgaria,Burkina Faso,Burundi,Cabo Verde,Cambodia,Cameroon,Canada,Cayman Islands,Central African Republic,Chad,Channel Islands,Chile,China,Colombia,Comoros,"Congo, Dem. Rep.","Congo, Rep.",Costa Rica,Cote d'Ivoire,Croatia,Cuba,Curacao,Cyprus,Czech Republic,Denmark,Djibouti,Dominica,Dominican Republic,Ecuador,"Egypt, Arab Rep.",El Salvador,Equatorial Guinea,Eritrea,Estonia,Eswatini,Ethiopia,Faroe Islands,Fiji,Finland,France,French Polynesia,Gabon,"Gambia, The",Georgia,Germany,Ghana,Gibraltar,Greece,Greenland,Grenada,Guam,Guatemala,Guinea,Guinea-Bissau,Guyana,Haiti,Honduras,"Hong Kong SAR, China",Hungary,Iceland,India,Indonesia,"Iran, Islamic Rep.",Iraq,Ireland,Isle of Man,Israel,Italy,Jamaica,Japan,Jordan,Kazakhstan,Kenya,Kiribati,"Korea, Dem. People’s Rep.","Korea, Rep.",Kosovo,Kuwait,Kyrgyz Republic,Lao PDR,Latvia,Lebanon,Lesotho,Liberia,Libya,Liechtenstein,Lithuania,Luxembourg,"Macao SAR, China",Madagascar,Malawi,Malaysia,Maldives,Mali,Malta,Marshall Islands,Mauritania,Mauritius,Mexico,"Micronesia, Fed. Sts.",Moldova,Monaco,Mongolia,Montenegro,Morocco,Mozambique,Myanmar,Namibia,Nauru,Nepal,Netherlands,New Caledonia,New Zealand,Nicaragua,Niger,Nigeria,Northern Mariana Islands,Norway,Oman,Pakistan,Palau,Panama,Papua New Guinea,Paraguay,Peru,Philippines,Poland,Portugal,Puerto Rico,Qatar,Romania,Russian Federation,Rwanda,Samoa,San Marino,Sao Tome and Principe,Saudi Arabia,Senegal,Serbia,Seychelles,Sierra Leone,Singapore,Sint Maarten (Dutch part),Slovak Republic,Slovenia,Solomon Islands,Somalia,South Africa,South Sudan,Spain,Sri Lanka,St. Kitts and Nevis,St. Lucia,St. Martin (French part),St. Vincent and the Grenadines,Sudan,Suriname,Sweden,Switzerland,Syrian Arab Republic,Tajikistan,Tanzania,Thailand,Timor-Leste,Togo,Tonga,Trinidad and Tobago,Tunisia,Turkey,Turkmenistan,Turks and Caicos Islands,Tuvalu,Uganda,Ukraine,United Arab Emirates,United Kingdom,United States,Uruguay,Uzbekistan,Vanuatu,"Venezuela, RB",Vietnam,Virgin Islands (U.S.),West Bank and Gaza,"Yemen, Rep.",Zambia,Zimbabwe
2000,2009376,308903,3118366,5752,6539,1644092,8358,3705745,306959,9085,1915300,801157,804860,29789,66461,13158124,26985,997961,1025125,24731,686595,6183,57342,833951,376671,172834,17528759,2064,33324,817017,1160794,640071,43508,1215235,1527423,3076970,4169,375499,834256,14872,1526275,126264500,4040396,54236,4707639,322573,392544,1668656,442600,1115074,13386,94329,1025506,533962,71758,6968,856262,1262860,6990599,586763,61432,339280,139698,106147,6653733,4726,81122,517621,6091250,23726,123112,123184,441830,8221151,1893876,3118,1080581,5620,10162,15533,1165074,880855,124323,75330,854920,652428,666500,1021097,28120,105305091,21154043,6613185,2356541,380517,7255,628900,5694211,265686,12684300,510313,1488363,3145048,8441,2292908,4700811,170000,205074,489840,532930,236755,323537,186870,288452,535575,3329,349954,43630,42798,1576681,1137617,2318561,28038,1096769,39009,5216,270936,118687,10171967,10743,36

### 5. Transpose the dataframe to match format of other dataframes

In [74]:
trans_pop = totalpop_df.melt(id_vars='Country Name')

In [77]:
trans_pop.rename(columns={'variable':'Year'}, inplace=True)
trans_pop.rename(columns={'value':'Population'}, inplace=True)
trans_pop.head()


,Country Name,Year,Population
0,Afghanistan,2000,2009376
1,Albania,2000,308903
2,Algeria,2000,3118366
3,American Samoa,2000,5752
4,Andorra,2000,6539


In [87]:
trans_pop.to_pickle('test.pkl')

In [88]:
t = pd.read_pickle('test.pkl')
t

,Country Name,Year,Population
0,Afghanistan,2000,2009376
1,Albania,2000,308903
2,Algeria,2000,3118366
3,American Samoa,2000,5752
4,Andorra,2000,6539
5,Angola,2000,1644092
6,Antigua and Barbuda,2000,8358
7,Argentina,2000,3705745
8,Armenia,2000,306959
9,Aruba,2000,9085


In [85]:
pickle.dump(trans_pop, open('population_df.pickle', 'wb'))